In [1]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf

2023-05-24 09:26:34.700282: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dev = qml.device('cirq.simulator',wires=3)

In [3]:
#@qml.qnode(dev)
def real(angles,**kwargs):
    qml.Hadamard(wires=0)
    qml.Rot(*angles,wires=0)
    #return qml.expval(qml.PauliZ(0))
    #return qml.probs(wires=0)
#real([0,0,0])

In [4]:
drawer = qml.draw(real)
w = ['phi', 'theta', 'omega']
print(drawer(w))

#cir = qml.QNode(real,dev)
#import matplotlib.pyplot as plt

#qml.drawer.use_style("black_white")
#fig, ax = qml.draw_mpl(cir)([np.pi/4,np.pi/4,np.pi/4])
#plt.show()

0: ──H──Rot(phi,theta,omega)─┤  


In [5]:
def generator(w,**kwargs):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)

In [6]:
drawer = qml.draw(generator,show_all_wires=True,wire_order=[0,1,2])
w = ['w0','w1','w2','w3','w4','w5','w6','w7','w8']
#w = ['w0','w1','w2']
print(drawer(w))

0: ──H───────RX(w0)──RY(w2)──RZ(w4)─╭●──RX(w6)──RY(w7)──RZ(w8)─┤  
1: ──RX(w1)──RY(w3)──RZ(w5)─────────╰X─────────────────────────┤  
2: ────────────────────────────────────────────────────────────┤  


In [7]:
def discriminator(w):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

In [8]:
drawer = qml.draw(discriminator,show_all_wires=True,wire_order=[0,1,2])
w = ['w0','w1','w2','w3','w4','w5','w6','w7','w8']
#w = ['w0','w1','w2']
print(drawer(w))

0: ──H───────RX(w0)──RY(w2)──RZ(w4)─╭●─────────────────────────┤  
1: ─────────────────────────────────│──────────────────────────┤  
2: ──RX(w1)──RY(w3)──RZ(w5)─────────╰X──RX(w6)──RY(w7)──RZ(w8)─┤  


In [10]:
@qml.qnode(dev, interface='tf')
def real_disc_circuit(phi, theta, omega, disc_weights):
    real([phi,theta,omega])
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

@qml.qnode(dev,interface='tf')
def gen_disc_circuit(gen_weights,disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

#cost_real = qml.QNode(real_disc_circuit, device)
#cost_gen = qml.QNode(gen_disc_circuit, device)

In [11]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi,theta,omega,disc_weights)
    #convert to probability
    prob_real_true = (true_disc_output+1)/2
    return prob_real_true

In [12]:
def prob_fake_true(gen_weights,disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights,disc_weights)
    #convert to probability
    prob_fake_true = (fake_disc_output+1)/2
    return prob_fake_true

In [13]:
def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights,disc_weights)-prob_real_true(disc_weights)
    return cost
def gen_cost(gen_weights):
    cost = -prob_fake_true(gen_weights,disc_weights)
    return cost

In [36]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))

init_disc_weights = np.random.normal(size=(9,))
gen_weights = tf.Variable(init_gen_weights,dtype=tf.float64)
disc_weights = tf.Variable(init_disc_weights,dtype=tf.float64)

print('disc_weights =',disc_weights)
print('loss =',disc_cost(disc_weights))
#print(gen_cost(gen_weights))
#print(prob_fake_true(gen_weights,disc_weights))


disc_weights = <tf.Variable 'Variable:0' shape=(9,) dtype=float64, numpy=
array([ 0.4105985 ,  0.14404357,  1.45427351,  0.76103773,  0.12167502,
        0.44386323,  0.33367433,  1.49407907, -0.20515826])>
loss = tf.Tensor(-0.02251306176185608, shape=(), dtype=float64)


In [37]:
opt = tf.keras.optimizers.experimental.SGD(learning_rate=0.4)

var = disc_weights
print('var =',var)

loss = lambda: disc_cost(disc_weights)         # d(loss)/d(var1) = var1
print('loss =',disc_cost(disc_weights) )

for step in range(50):
    opt.minimize(loss, [var])
    if step % 5 == 0:
        loss_val = loss().numpy()
        print("Step {}: cost = {}".format(step, loss_val))
        

var = <tf.Variable 'Variable:0' shape=(9,) dtype=float64, numpy=
array([ 0.4105985 ,  0.14404357,  1.45427351,  0.76103773,  0.12167502,
        0.44386323,  0.33367433,  1.49407907, -0.20515826])>
loss = tf.Tensor(-0.02251306176185608, shape=(), dtype=float64)
Step 0: cost = -0.05727693438529968
Step 5: cost = -0.2634810581803322
Step 10: cost = -0.42739150300621986
Step 15: cost = -0.4726157411932945
Step 20: cost = -0.48406897485256195
Step 25: cost = -0.48946385085582733
Step 30: cost = -0.4928187355399132
Step 35: cost = -0.49494922906160355
Step 40: cost = -0.4962702915072441
Step 45: cost = -0.4970718249678612


In [38]:
print("Prob(real classified as real): ", prob_real_true(var).numpy())

Prob(real classified as real):  0.9985871016979218


In [39]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, var).numpy())

Prob(fake classified as real):  0.5011128559708595


In [40]:
opt = tf.keras.optimizers.legacy.SGD(learning_rate=0.4)
var_g = gen_weights
print('var_g =',var_g)

loss = lambda: gen_cost(gen_weights)         # d(loss)/d(var1) = var1
print('loss =',gen_cost(gen_weights))

for step in range(50):
    opt.minimize(loss, [var_g])
    if step % 5 == 0:
        loss_val = loss().numpy()
        print("Step {}: cost = {}".format(step, loss_val))
        

var_g = <tf.Variable 'Variable:0' shape=(9,) dtype=float64, numpy=
array([ 3.15923318e+00,  4.00157208e-03,  9.78737984e-03,  2.24089320e-02,
        1.86755799e-02, -9.77277880e-03,  9.50088418e-03, -1.51357208e-03,
       -1.03218852e-03])>
loss = tf.Tensor(-0.5011128559708595, shape=(), dtype=float64)
Step 0: cost = -0.5833386555314064
Step 5: cost = -0.8915733397006989
Step 10: cost = -0.9784241914749146
Step 15: cost = -0.9946482479572296
Step 20: cost = -0.9984995126724243
Step 25: cost = -0.9995638430118561
Step 30: cost = -0.9998717308044434
Step 35: cost = -0.9999619722366333
Step 40: cost = -0.9999887645244598
Step 45: cost = -0.9999964833259583


In [41]:
print("Prob(fake classified as real): ", prob_fake_true(var_g, var).numpy())

Prob(fake classified as real):  0.9999986886978149


In [42]:
print("Discriminator cost: ", disc_cost(var).numpy())

Discriminator cost:  0.0014115869998931885


In [43]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

@qml.qnode(dev, interface="tf")
def bloch_vector_real(angles):
    real(angles)
    return [qml.expval(o) for o in obs]

@qml.qnode(dev, interface="tf")
def bloch_vector_generator(angles):
    generator(angles)
    return [qml.expval(o) for o in obs]

print(f"Real Bloch vector: {bloch_vector_real([phi, theta, omega])}")
print(f"Generator Bloch vector: {bloch_vector_generator(var_g)}")

Real Bloch vector: [<tf.Tensor: shape=(), dtype=float64, numpy=-0.21694186329841614>, <tf.Tensor: shape=(), dtype=float64, numpy=0.45048442482948303>, <tf.Tensor: shape=(), dtype=float64, numpy=-0.8660252094268799>]
Generator Bloch vector: [<tf.Tensor: shape=(), dtype=float64, numpy=-0.28404659032821655>, <tf.Tensor: shape=(), dtype=float64, numpy=0.4189322590827942>, <tf.Tensor: shape=(), dtype=float64, numpy=-0.8624440431594849>]


In [44]:
qml.math.fidelity(bloch_vector_real([phi, theta, omega]), bloch_vector_generator(var_g))

0.9944922795846928